# Random Survival Forests

In [1]:
pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import set_config
from sklearn.model_selection import RepeatedKFold, GridSearchCV

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.ensemble import RandomSurvivalForest

set_config(display="text")  # displays text representation of estimators
sns.set_style("whitegrid")

In [4]:
# load dataset
data_x, y = load_veterans_lung_cancer()
x = encode_categorical(data_x)

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
rsf = RandomSurvivalForest(
    n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=42
)
rsf.fit(x_train, y_train)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10,
                     n_estimators=1000, n_jobs=-1, random_state=42)

In [ ]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y["Status"], y["Survival_in_days"], -prediction)
    return result[0]

In [ ]:
param_grid = {"alpha": 2.0 ** np.arange(-12, 13, 2)}

# 5 repetitions of 5-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
gcv = GridSearchCV(rsf, param_grid, scoring=score_survival_model, n_jobs=1, refit=False, cv=cv)

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
gcv = gcv.fit(x_train, y_train)

In [ ]:
print("Best parameters:", gcv.best_params_)
print("Best C-index:", round(gcv.best_score_, 3))

In [ ]:
def plot_performance(gcv):
    n_splits = gcv.cv.get_n_splits()
    cv_results = pd.DataFrame(gcv.cv_results_)
    cv_results = cv_results.rename(columns=lambda x: x.replace("param_", ""))

    plt.figure(figsize=(6, 3))
    sns.stripplot(x="alpha", y="mean_test_score", data=cv_results, jitter=True, dodge=True)
    plt.title("Cross-validated Performance vs. Alpha")
    plt.xlabel("Alpha")
    plt.ylabel("Mean Evaluation Score")
    plt.xticks(rotation=45)
    plt.show()

plot_performance(gcv)

In [ ]:
estimator.set_params(**gcv.best_params_)
estimator.fit(x_train, y_train)

In [ ]:
test_pred = estimator.predict(x_test)
#print(np.round(test_pred, 3))
#print(y_test)

# Evaluate the model on the test set
test_cindex = concordance_index_censored(
    y_test["Status"],
    y_test["Survival_in_days"],
    -test_pred)           # flip sign to obtain risk scores
print("C-index on test set:", round(test_cindex[0], 3))


In [ ]:
# reference estimator (regression) and result

ref_estimator = FastSurvivalSVM(rank_ratio=0.0, max_iter=1000, tol=1e-5, random_state=42)
ref_estimator.fit(x_train, y_train)

cindex = concordance_index_censored(
    y_train["Status"],
    y_train["Survival_in_days"],
    -ref_estimator.predict(x_train),  # flip sign to obtain risk scores
)
print("C-index (train):", round(cindex[0], 3))

cindex = concordance_index_censored(
    y_test["Status"],
    y_test["Survival_in_days"],
    -ref_estimator.predict(x_test),  # flip sign to obtain risk scores
)
print("C-index (test):", round(cindex[0], 3))

In [ ]:
pred = ref_estimator.predict(x_test.iloc[:2])
print(np.round(pred, 3))